In [4]:
import os
import h5py
from keras import backend as K

In [5]:
import numpy as np
import argparse
from path import Path

from keras.models import Model
from keras.layers import Dense, Dropout
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from data_loader import train_generator, val_generator

Train set size :  (450,) (450, 5)
Val set size :  (50,) (50, 5)
Train and validation datasets ready !


In [6]:
class TensorBoardBatch(TensorBoard):
    def __init__(self, *args, **kwargs):
        super(TensorBoardBatch, self).__init__(*args)

        # conditionally import tensorflow iff TensorBoardBatch is created
        self.tf = __import__('tensorflow')

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}

        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = self.tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(summary, batch)

        self.writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = self.tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(summary, epoch * self.batch_size)

        self.writer.flush()

In [7]:
def earth_mover_loss(y_true, y_pred):
    cdf_ytrue = K.cumsum(y_true, axis=-1)
    cdf_ypred = K.cumsum(y_pred, axis=-1)
    samplewise_emd = K.sqrt(K.mean(K.square(K.abs(cdf_ytrue - cdf_ypred)), axis=-1))
    return K.mean(samplewise_emd)

In [18]:
# load learned mobilenet 
weights_path = './neural-image-assessment/weights/mobilenet_weights.h5'

base_model = MobileNet((None, None, 3), alpha=1, include_top=False, pooling='avg', weights=None)
x = Dropout(0.75)(base_model.output)
x = Dense(5, activation='softmax', kernel_initializer='uniform')(x)
model = Model(base_model.input, x)
model.load_weights(weights_path, by_name=True)

# freeze ~88
for layer in model.layers[:79]:
    layer.trainable = False

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (Activation)      (None, None, None, 32)    0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
__________

In [24]:
optimizer = Adam(lr=1e-3)
model.compile(optimizer, loss=earth_mover_loss)

checkpoint = ModelCheckpoint('./neural-image-assessment/weights/mobilenet_weights.h5', monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True,
                             mode='min')
tensorboard = TensorBoardBatch()
callbacks = [checkpoint, tensorboard]

batchsize = 500
epochs = 50

model.fit_generator(train_generator(batchsize=batchsize),
                    steps_per_epoch=1,
                    epochs=epochs, verbose=1, callbacks=callbacks,
                    validation_data=val_generator(batchsize=batchsize),
                    validation_steps=1)

Epoch 1/50
1/1 [==============================] - 9s 9s/step - loss: 0.2431 - val_loss: 0.3345

Epoch 00001: val_loss improved from inf to 0.33449, saving model to ./neural-image-assessment/weights/mobilenet_weights.h5
Epoch 2/50
1/1 [==============================] - 1s 906ms/step - loss: 0.2489 - val_loss: 0.3057

Epoch 00002: val_loss improved from 0.33449 to 0.30570, saving model to ./neural-image-assessment/weights/mobilenet_weights.h5
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 0.2514 - val_loss: 0.3147

Epoch 00003: val_loss did not improve from 0.30570
Epoch 4/50
1/1 [==============================] - 2s 2s/step - loss: 0.2517 - val_loss: 0.3166

Epoch 00004: val_loss did not improve from 0.30570
Epoch 5/50
1/1 [==============================] - 2s 2s/step - loss: 0.2461 - val_loss: 0.3090

Epoch 00005: val_loss did not improve from 0.30570
Epoch 6/50
1/1 [==============================] - 2s 2s/step - loss: 0.2507 - val_loss: 0.2908

Epoch 00006: val_l

Exception ignored in: <generator object val_generator at 0x7f1e9f21e4c0>
Traceback (most recent call last):
  File "/home/wakacho/jphacks/instagenic_regressor/instagenic_regressor/data_loader.py", line 110, in val_generator
    yield (X_batch, y_batch)
  File "/home/wakacho/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1596, in __exit__
    self._default_graph_context_manager.__exit__(exec_type, exec_value, exec_tb)
  File "/home/wakacho/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/home/wakacho/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 5267, in get_controller
    context.context().context_switches.pop()
  File "/home/wakacho/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/eager/context.py", line 136, in pop
    self.stack.pop()
IndexError: pop from empt

In [20]:
def predict(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    scores = model.predict(x, batch_size=1, verbose=0)[0]
    
    return scores

In [25]:
for filename in ['BJzcrQFjfsQ.jpg',
 'BlSfwPHB5a_.jpg',
 'BlsoWzlFesk.jpg',
 'BmNTJPdHFLI.jpg',
 'Bn-l8xXA1oe.jpg',
 'Bn-mgscAGfH.jpg',
 'Bo1lDTCBL9j.jpg',
 'BoMUTAphCAL.jpg',
 'BoqI5d9h1pi.jpg',
 'BpEQqVMgzRJ.jpg',
 'BpEyhasA9sF.jpg',
 'BpFFLS-hIPb.jpg',
 'BpGUO3IF7af.jpg',
 'BpJv95dnC3n.jpg',
 'BpO-0nKBR3g.jpg',
 'BpO-2YOHIml.jpg',
 'BpO-36fh5z1.jpg',
 'BpO-68Qn_LO.jpg',
 'BpO-AYqFD-9.jpg',
 'BpO-KZQgOq9.jpg']:
    print(filename)
    print(np.argmax(predict('../images/omelette_rice_500/images/'+filename)))

BJzcrQFjfsQ.jpg
1
BlSfwPHB5a_.jpg
1
BlsoWzlFesk.jpg
1
BmNTJPdHFLI.jpg
1
Bn-l8xXA1oe.jpg
1
Bn-mgscAGfH.jpg
1
Bo1lDTCBL9j.jpg
1
BoMUTAphCAL.jpg
1
BoqI5d9h1pi.jpg
1
BpEQqVMgzRJ.jpg
1
BpEyhasA9sF.jpg
1
BpFFLS-hIPb.jpg
1
BpGUO3IF7af.jpg
1
BpJv95dnC3n.jpg
1
BpO-0nKBR3g.jpg
1
BpO-2YOHIml.jpg
1
BpO-36fh5z1.jpg
1
BpO-68Qn_LO.jpg
2
BpO-AYqFD-9.jpg
1
BpO-KZQgOq9.jpg
1
